# Inteligencia Artificial
# - Laboratorio 8 -

Integrantes
- Mark Albrand 
- Jimena Hernández
- Melissa Perez

## Tasks 1 - Teoría

### 1. Investigar el algoritmo AC-3 y su relación con el algoritmo de backtracking search

``` text
The AC-3 Algorithm
1: Put (v, C) in the set S for every variable v and every constraint involving v.
2: while S is not empty do
3: remove (Xi, Cij) from S (Cij is a constraint between Xi and Xj.)
4: if Revise(Xi, Cij) then
5: if dom(Xi) is empty then return false
6: for Xk where Cki is a constraint between Xk and Xi do
7: add (Xk, Cki) to S
8: end for
9: end if
10: end while
11: return true 
```
Este algoritmo funciona sobre variables, sus dominios y las restricciones entre ellas. Este usa el arc consistency entre pares de variables, va eliminando valores del dominio de una variable que no este alineada con las restricciones impuestas por otra variables. Esto se puede ver reflejado en la línea 3. Este proceso se repite hasta que no se pueden eliminar más valores, o se encuentra una inconsistencia. Este algoritmo es eficaz ya que reduce el espacio de busqueda antes o durante la busqueda de soluciones.

La relación que tiene con el backtracking search ya que implementar dicho algoritmo junto a bactracking search mejora significativamente la eficiencia del proceso de búsqueda, esto debido a que disminuye el número de posibilidades que el algoritmo de backtracking necesita explorar.

Gao, A. (s.f.). Constraint Satisfaction Problems: Backtracking Search and Arc Consistency. Lecture 5. Basado en el trabajo de K. Leyton-Brown, K. Larson, y P. van Beek. Recuperado de https://cs.uwaterloo.ca/~a23gao/cs486686_s19/slides/lec05_csp_arc_consistency_backtracking_search_nosol.pdf

### 2. Defina en sus propias palabras el término “Arc Consistency”

Esta consiste en una técnica de propagación para restricciones binarias. Para cada valor de una variable en la restricción, buscamos un valor de soporte que se pueda asignar a la otra variable. Si no hay ninguno, el valor puede ser eliminado de forma segura. De lo contrario, la restricción si cuenta con arc consistency. Basicamente buscamos valores que obviamente pueden ser eliminados al considerar una única restricción de manera aislada. Si hemos eliminado todos esos valores, entonces todo es consistente.


## Tasks 2 - CSP con Backtracking, Beam y Local Search

Implementar tres algoritmos diferentes de satisfacción de restricciones para resolver un problema de programación de exámenes para cuatro estudiantes que toman siete exámenes diferentes. El
problema implica calendarizar los exámenes para los estudiantes respetando diversas limitaciones y preferencias.

Restricciones:
- Todos los exámenes deberán realizarse en días diferentes, concretamente lunes, martes y miércoles.
- Ningún estudiante deberá tener más de un examen por día.
- Los estudiantes que toman el mismo curso no pueden tener exámenes programados para el mismo día

### Definición de variables

In [12]:
exams = {"Exam 1", "Exam 2", "Exam 3", "Exam 4", "Exam 5", "Exam 6", "Exam 7"}
students = {"Student 1", "Student 2", "Student 3", "Student 4"}
days = {"Monday", "Tuesday", "Wednesday"}

schedule = {d: {} for d in days}
print(schedule)

# metricas de tiempo de ejecucion
from time import time

{'Monday': {}, 'Tuesday': {}, 'Wednesday': {}}


### Backtracking Search

In [11]:
def backtracking(schedule, exams, students, days):
    def is_valid(schedule, exam, student, day):
        if student in schedule[day].values():
            return False
        if day in schedule and exam in schedule[day]:
            return False
        return True

    if not exams:
        return True
    exam = exams.pop()
    for student in students:
        for day in days:
            if is_valid(schedule, exam, student, day):
                schedule[day][exam] = student
                if backtracking(schedule, exams, students, days):
                    return True
                del schedule[day][exam]
    exams.add(exam)
    return False

time_start_backtracking = time()
backtracking(schedule, exams, students, days)
time_end_backtracking = time()

time_diff = (time_end_backtracking - time_start_backtracking) * 1000
print(f"Tiempo de ejecución Backtracking search: {time_diff} milisegundos")

for day, exams in schedule.items():
    print(day)
    for exam, student in exams.items():
        print(f"\t{exam}: {student}")

Tiempo de ejecución Backtracking search: 0.05412101745605469 milisegundos
Monday
	Exam 5: Student 1
	Exam 1: Student 2
	Exam 6: Student 3
Tuesday
	Exam 4: Student 1
	Exam 2: Student 2
Wednesday
	Exam 7: Student 1
	Exam 3: Student 2


### Beam Search

In [13]:
def beam_search(schedule, exams, students, days, beam_width):
    def is_valid(schedule, exam, student, day):
        if student in schedule[day].values():
            return False
        if day in schedule and exam in schedule[day]:
            return False
        return True

    def heuristic(schedule):
        """
        Define la heurística para evaluar la calidad de una solución.
        El valor retornado representa la cantidad de conflictos en el horario.
        """
        return sum(len(set(schedule[day].values())) - len(schedule[day]) for day in days)

    if not exams:
        return True

    exam = exams.pop()
    candidates = []
    for student in students:
        for day in days:
            if is_valid(schedule, exam, student, day):
                schedule[day][exam] = student
                candidates.append((day, student))
                del schedule[day][exam]

    candidates.sort(key=lambda x: heuristic(schedule))
    candidates = candidates[:beam_width]

    for day, student in candidates:
        schedule[day][exam] = student
        if beam_search(schedule, exams, students, days, beam_width):
            return True
        del schedule[day][exam]

    exams.add(exam)
    return False

beam_width = 2

time_start_beam_search = time()
success = beam_search(schedule, exams, students, days, beam_width)
time_end_beam_search = time()

time_diff = (time_end_beam_search - time_start_beam_search) * 1000
print(f"Tiempo de ejecución Beam search: {time_diff} milisegundos")

if success:
    for day, exams in schedule.items():
        print(day)
        for exam, student in exams.items():
            print(f"\t{exam}: {student}")
else:
    print("No se encontró solución.")

Tiempo de ejecución Beam search: 0.17380714416503906 milisegundos
Monday
	Exam 5: Student 1
	Exam 1: Student 2
	Exam 6: Student 3
Tuesday
	Exam 4: Student 1
	Exam 2: Student 2
Wednesday
	Exam 7: Student 1
	Exam 3: Student 2


### Local search

In [16]:
import random

exams = {"Exam 1", "Exam 2", "Exam 3", "Exam 4", "Exam 5", "Exam 6", "Exam 7"}
students = {"Student 1", "Student 2", "Student 3", "Student 4"}
days = {"Monday", "Tuesday", "Wednesday"}

schedule = {d: {} for d in days}

def local_search(schedule, exams, students, days, max_iterations):
    def is_valid(schedule, exam, student, day):
        if student in schedule[day].values():
            return False
        if day in schedule and exam in schedule[day]:
            return False
        return True

    def select_best_move(schedule):
        best_move = None
        best_delta = float('inf')
        for source_day in days:
            for target_day in days:
                for exam, student in schedule[source_day].items():
                    if is_valid(schedule, exam, student, target_day):
                        delta_conflicts = 0
                        if source_day != target_day:
                            delta_conflicts -= 1
                        for other_exam, other_student in schedule[target_day].items():
                            if other_exam != exam and other_student == student:
                                delta_conflicts += 1
                        if delta_conflicts < best_delta:
                            best_move = (exam, source_day, target_day)
                            best_delta = delta_conflicts
        return best_move

    for exam in exams:
        day = random.choice(days)
        student = random.choice(list(students))
        schedule[day][exam] = student

    for _ in range(max_iterations):
        move = select_best_move(schedule)
        if move is None:
            break
        exam, source_day, target_day = move
        student = schedule[source_day][exam]
        del schedule[source_day][exam]
        schedule[target_day][exam] = student

    return schedule

schedule = {day: {} for day in days}
days = list(days)

max_iterations = 1000

time_start_local_search = time()
new_schedule = local_search(schedule, exams, students, days, max_iterations)
time_end_local_search = time()

time_diff = (time_end_local_search - time_start_local_search) * 1000
print(f"Tiempo de ejecución local search: {time_diff} milisegundos")

for day, exams in new_schedule.items():
    print(day)
    for exam, student in exams.items():
        print(f"\t{exam}: {student}")

Tiempo de ejecución local search: 7.447004318237305 milisegundos
Monday
	Exam 4: Student 3
Tuesday
	Exam 3: Student 1
	Exam 6: Student 4
Wednesday
	Exam 7: Student 3
	Exam 1: Student 2
	Exam 5: Student 4
	Exam 2: Student 1


## Conclusiones

- En cuanto a los tiempos de ejecución, se puede ver que el algoritmo de backtracking y el algoritmo de beam search son más rápidos que el algoritmo de local search. Ambos con tiempos de ejecución rondando de 0.05 a 0.1 milisegundos, mientras que el algoritmo de local search tiene un tiempos de ejecución de 6.0 milisegundos. 
- Por otro lado, el algoritmo de local search es el que tiene más iteraciones, mientras que el algoritmo de backtracking es el que tiene menos iteraciones.
- El algoritmo de local search desarrollado provee respuestas más robustas, populando los 3 días con exámenes, mientras que los otros algoritmos no siempre logran esto (pero siempre cumpliendo con las restricciones propuestas).